In [ ]:
using OffsetArrays # -> Nos permitirá usar matrices que empiecen con índice 0, es lento pero cómodo en este caso
using DataStructures # -> definir una cola de prioridad

# Teoría de grafos

Vamos a ver el algoritmo de Djikstra

Implementación

P1. <b>Inicialización</b>: l(s):=0, l(v):=$\infty$ para todo $v\in V \setminus \{s\}$, R:=$\emptyset$

P2. <b>Buscar Probe</b>: Buscar un vértice $v\in V \setminus R$ tal que $l(v):=\min_{w\in V\setminus R} l(w)$

P3. <b>Update R</b>: Sea $R:=R\cup \{v\}$

P4. actualizar:

Para todo $w\in V \setminus R$ tal que $(v,w)\in A$ hacer:

$~~~Si $l(w)&gt;l(v)+c(v,w)$ entonces 

$~~~~~~$sea $l(w):=l(v)+c(v,w)$ y $\pi(w):=v$ 

$~~~$Fin Si 

Fin Para 

P5. <b>Condición de final</b>: Si $R\neq V$ ir a paso 2.

Aunque este algoritmo puede implementarse sin cambios en Julia, vamos a ver una versión alternativa que usará una cola de prioridad (que acostumbrará a ser más eficiente en la práctica) 

In [ ]:
type grafoDenso
    vertices::Int64
    arcos::Array{Int64,2} #arco[i,j] indica la distancia entre i y j 
end

# toma un grafo (llamado grafo) y un vértice origen (llamado origen)
# devolver el camino
function djDense(grafo::grafoDenso,origen::Int64)
    #inicializa el problema
    ∞=1000000 #la distancia máxima entre el orgen y cualquier vértice
    π=zeros(Int64,grafo.vertices) # el vector pi guarda el último vértice usado para llegar a otro vértice
    # π[3] cuál es el último vértice que se visita en el camino entre origen y 3 
    l=zeros(Int64,grafo.vertices) # el vector l guarda la longitud del camino
    # l[3] es la longitud del camino entre origen y 3
    fill!(l,∞) #distancia entre origen y cualquier otro vértice igual a infinito
    l[origen]=0 #excepto origen
    #fin inicializar el problema

    #¿puedo mejorar un camino desde algún vértice?
    # Q contiene los vértices para los que la respuesta es afirmativa
    Q=PriorityQueue() #cola con prioridad
    #puedo mejorar un vértice desde el origen
    enqueue!(Q,origen,0) #la cola de prioridad Q guarda los vértices desde que se puede mejorar algún cambio
    while isempty(Q)==false #no va a haber más mejoras
        v=dequeue!(Q) #sacar un vértice de Q para desarrollar caminos
        #println("*** uso el vértice ",v," para encontrar caminos")
        for w in 1:grafo.vertices #ver todos los arcos que salen de ese vértice
            #todos los arcos (v,w) si no hay arco grafo.arcos[v,w]=10.000, no se escogerá
            if l[w] > (l[v]+grafo.arcos[v,w]) #si mejora la distanmcia de algún vértice
                #println("he encontrado un mejor camino para llegar a ",w," pasando por ",v)
                l[w]=l[v]+grafo.arcos[v,w] #actualizo la longitud que hay
                π[w]=v
                if haskey(Q,w)
                    Q[w]=l[w]
                else
                    enqueue!(Q,w,l[w])
                end
            end
        end
    end
    return l,π
end

In [ ]:
function generarGrafoDense(n::Int64,maxValue::Int64)
    a=Array{Int64}(n,n)
    rand!(a,1:maxValue)
    for i in 1:n
        a[i,i]=0
    end
    return grafoDenso(n,a) 
end

#unGrafo=generarGrafoDense(1000,1000)

z=10000
ejemplo=grafoDenso(7,[0 7 z 2 z z z; z 0 z z 2 z z; 1 z 0 2 z z z; z z z 0 2 2 z; z z z 2 0 z 3; z z z z z 0 3; z z z z z 3 0])
println(ejemplo)
longitud,traza=djDense(ejemplo,3)
#println(unGrafo.arcos)
println("longitud: ",longitud)
println("traza: ",traza)

Generalmente es posible aprovecharse de las características del grafo. Veremos aquí el caso de los grafos sparse que complicarán un tanto la implementación a cambio de mejoras significativas en la eficiencia

In [ ]:
g=generarGrafoDense(10000,1000)
@time(djDense(g,1))

In [ ]:
type arco
    origen::Int64
    destino::Int64
    longitud::Int64
end

type grafoSparse
    nVertices::Int64
    nArcos::Int64
    pOrigen::Array{Int64,1}
    arcos::Array{arco,1} #arco[i,j] indica la distancia entre i y j 
end

function djSparse(origen::Int64,g::grafoSparse)
    ∞=1000000
    π=zeros(Int64,g.nVertices)
    l=Array{Int64}(g.nVertices)
    fill!(l,∞)
    l[origen]=0
    Q=PriorityQueue()
    enqueue!(Q,origen,0)
    while isempty(Q)==false
        v=dequeue!(Q)
        for c in g.pOrigen[v]:g.pOrigen[v+1]-1 #cambio principal
            w=g.arcos[c].destino
            longitud=g.arcos[c].longitud
            if l[w] > (l[v]+longitud)
                l[w]=l[v]+longitud
                π[w]=v
                if haskey(Q,w)
                    Q[w]=l[w]
                else
                    enqueue!(Q,w,l[w])
                end
            end
        end
    end
    return l,π
end



In [ ]:
function generarGrafoSparse(n::Int64,p::Float64,maxValue::Int64)
    a=arco[]
    pOrigen=Int64[]
    nArcos=1
    for i in 1:n
        push!(pOrigen,nArcos)
        for j in 1:n
            if i!=j
                if rand()<p
                    push!(a,arco(i,j,rand(1:maxValue)))
                    nArcos += 1
                end
            end
        end
    end
    push!(pOrigen,nArcos)
    return grafoSparse(n,nArcos-1,pOrigen,a) 
end

unGrafo=generarGrafoSparse(10,0.25,1000)
println(unGrafo.pOrigen)
println(unGrafo.arcos)
longitud,camino=djSparse(1,unGrafo)
println(longitud)
println(camino)

In [ ]:
grafTest=generarGrafoSparse(10000,0.01,10000)
@time(djSparse(1,grafTest))

# Programación dinámica

Vamos a ver un ejemplo muy simple de programación dinámica, el problema de knapsack.

El problema de knapsack es posiblemente uno de los modelos clásicos más simples que hay. Ofrece una formulación de programación entera simple que además ha sido muy tratada en la literatura. 

Trataremos la versión 0-1. En la formulación propuesta, cada etapa (columna de la tabla) corresponde a un item. Los estados representan la disponibilidad (beneficio máximo posible con una combinación de los elementos hasta esa etapa usando la capacidad indicada en la fila de la tabla).

No se muestra cómo reconstruir la solución.

In [ ]:
beneficio = [ 5, 3, 2, 7, 4 ]
peso = [ 2, 8, 4, 2, 5 ]
capacidad = 10

In [ ]:
function knapsack(b,p,c)
    T=size(p,1) #número de etapas
    tabla = OffsetArray(Int64, 0:c, 1:T) #las filas representan la capacidad ocupada, las columnas las decisiones
    #etapa 1
    for s=1:c
        tabla[s,1]=(-1) #valor negativo va a indicar que no es valor posible
    end
    tabla[0,1]=0
    tabla[p[1],1]=b[1]
    #etapas 2 a T
    println(tabla[:,1]) #--> permitirá explicar el funcionamiento del algoritmo
    for t in 2:T
        for s=0:c
            tabla[s,t]=tabla[s,t-1]
        end
        for s=c-p[t]:-1:0 #este for va de c-p[t] hasta 0 reduciendo en 1 cada vez
            if tabla[s,t]>=0 #contiene un valor posible 
                if tabla[s+p[t],t]<(tabla[s,t-1]+b[t]) #mejora la opción actual?
                    tabla[s+p[t],t]=tabla[s,t-1]+b[t]
                end
            end
        end
        println(tabla[:,t]) #--> permitirá explicar el funcionamiento del algoritmo
    end
    beneficio=0
    maxLoad=0
    previous=0
    for s in 0:c
        if beneficio < tabla[s,T]
            beneficio=tabla[s,T]
        end
    end
    return beneficio
end

In [ ]:
beneficio=knapsack(beneficio,peso,capacidad)